- Usar otros Clasificadores

In [1]:
import pandas as pd
from glob import glob
import winsound
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

from sklearn.tree import DecisionTreeClassifier

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

token = RegexpTokenizer(r'[a-zA-Z0-9]+')
count_vect = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)

In [3]:
#fnames = glob("datasets/*_train.json")
#fnames = glob("datasets/meoffendes2021_mx_Es_train.json")
fnames = glob("datasets/haha2018_Es_train.json")
fnames.sort()

fn = fnames[0]
df = pd.read_json(fn, lines=True)
train_df = pd.read_json(fn, lines=True)
test_df = pd.read_json(fn.replace("_train", "_test"), lines=True)
train = train_df.loc[0:99, ['text', 'klass']]
test = test_df.loc[0:39, ['text', 'klass']]
train.shape


(100, 2)

In [4]:
P = []
for fn in fnames:

    ds_n = fn.replace("datasets",'').replace('_train.json','')
    print('__', ds_n, '___')

    train_df = pd.read_json(fn, lines=True)
    test_df = pd.read_json(fn.replace("_train", "_test"), lines=True)

    # recortados para pruebas
    train_df = train_df.loc[0:5999, ['text', 'klass']]
    test_df = test_df.loc[0:2999, ['text', 'klass']]
    
    ############
    ## Cambiar True/False para usar o no las BiFrases
    ###########
    if False:
        """
        TODO: en cada append, el dataset se reconstruye completo. Encontrar otro método más rápido.
        """
        _ = train_df.iloc[0]
        bi_train_df = pd.DataFrame([_], columns = ['text', 'klass'])
        for index, row in train_df.iloc[1:].iterrows():
            try:
                bi_train_df =  bi_train_df.append(row)
                if _['klass'] == row['klass']:
                    bi_frase = {'text': _['text'] + ' ' + row['text'], 'klass':row['klass']}
                    bi_train_df = bi_train_df.append(bi_frase, ignore_index=True)
                    _ =  {'text': '_V_A_C_I_A_', 'klass': None}
                else:   
                    _ = row
            except Exception as exc:
                print(exc)
                _ = row
                continue
        ('train_df Bi> ', train_df.shape, ' > ', bi_train_df.shape)
        train_df = bi_train_df
        winsound.Beep(3000, 900)

    print('train_df > ', train_df.shape)
    print('Test > ', test_df.shape)
    
    X_train = train_df[['text']]
    y_train = train_df[['klass']]
    
    X_test = test_df[['text']]
    y_test = test_df[['klass']]
    
    
    X_train_counts = count_vect.fit_transform(X_train['text'])    
    MNB = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)
    MNB.fit(X_train_counts, y_train.values.ravel())
    X_test_counts = count_vect.transform(X_test['text'])
    predicted = MNB.predict(X_test_counts)
    
    
    '''
    v = TfidfVectorizer(use_idf = True)
    x = v.fit_transform(X_train['text']).toarray()
    GNB = GaussianNB()
    GNB.fit(x, y_train.values.ravel())
    y = v.transform(X_test['text']).toarray()
    predicted = GNB.predict(y)
    '''
    
    '''
    v = TfidfVectorizer(use_idf = True)
    x = v.fit_transform(X_train['text']).toarray()
    DT = DecisionTreeClassifier()
    DT.fit(x, y_train)
    y = v.transform(X_test['text']).toarray()
    predicted = DT.predict(y)
    '''
    
    accuracy = metrics.accuracy_score(y_test, predicted)
    f1_score = metrics.f1_score(y_test, predicted, average="macro")
    precision_score = metrics.precision_score(y_test, predicted, average="macro")
    recall_score = metrics.recall_score(y_test, predicted, average="macro")
    f1_score_micro = metrics.recall_score(y_test, predicted, average="micro")
    
    p = [ds_n, accuracy, f1_score, precision_score, recall_score, f1_score_micro] 
    P.append(p)

    winsound.Beep(3000, 900)
    
    #break

#save_model(P, 'MNB_BiFrases_True.scores')    
    
winsound.Beep(1500, 1000) 

print('dataset,', 'accuracy,', 'f1_macro,', 'precision_macro,', 'recall_macro,', 'f1_micro')
for p in P:
    print("{0}, {1:1.5f}, {2:1.5f}, {3:1.5f}, {4:1.5f}, {5:1.5f}".format(p[0], p[1], p[2], p[3], p[4], p[5]))
    

__ /haha2018_Es ___
train_df >  (6000, 2)
Test >  (3000, 2)
dataset, accuracy, f1_macro, precision_macro, recall_macro, f1_micro
/haha2018_Es, 0.74000, 0.73096, 0.72870, 0.74224, 0.74000


In [5]:
# FALSE
print('train_df > ', train_df.shape)
print('Test > ', test_df.shape)
print('dataset,', 'accuracy,', 'f1_macro,', 'precision_macro,', 'recall_macro,', 'f1_micro')
for p in P:
    print("{0}, {1:1.5f}, {2:1.5f}, {3:1.5f}, {4:1.5f}, {5:1.5f}".format(p[0], p[1], p[2], p[3], p[4], p[5]))
print(classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))

train_df >  (6000, 2)
Test >  (3000, 2)
dataset, accuracy, f1_macro, precision_macro, recall_macro, f1_micro
/haha2018_Es, 0.74000, 0.73096, 0.72870, 0.74224, 0.74000
              precision    recall  f1-score   support

           0       0.83      0.73      0.78      1888
           1       0.62      0.75      0.68      1112

    accuracy                           0.74      3000
   macro avg       0.73      0.74      0.73      3000
weighted avg       0.76      0.74      0.74      3000

[[1385  503]
 [ 277  835]]


In [6]:
# TRUE
print('train_df > ', train_df.shape)
print('Test > ', test_df.shape)
print('dataset,', 'accuracy,', 'f1_macro,', 'precision_macro,', 'recall_macro,', 'f1_micro')
for p in P:
    print("{0}, {1:1.5f}, {2:1.5f}, {3:1.5f}, {4:1.5f}, {5:1.5f}".format(p[0], p[1], p[2], p[3], p[4], p[5]))

print(classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))

train_df >  (6000, 2)
Test >  (3000, 2)
dataset, accuracy, f1_macro, precision_macro, recall_macro, f1_micro
/haha2018_Es, 0.74000, 0.73096, 0.72870, 0.74224, 0.74000
              precision    recall  f1-score   support

           0       0.83      0.73      0.78      1888
           1       0.62      0.75      0.68      1112

    accuracy                           0.74      3000
   macro avg       0.73      0.74      0.73      3000
weighted avg       0.76      0.74      0.74      3000

[[1385  503]
 [ 277  835]]
